In [ ]:
import pandas as pd
import networkx as nx

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


# Creating a graph with ESCO dataset, and then creating sentence pairs with the ESCO titles and skills

In [ ]:
def build_esco_knowledge_graph():
    df_occupations = pd.read_csv("/content/drive/MyDrive/occupations_fr.csv")
    df_skills = pd.read_csv("/content/drive/MyDrive/skills_fr.csv")
    df_relations = pd.read_csv("/content/drive/MyDrive/occupationSkillRelations_fr.csv")
    print("Successfully loaded all CSV files.")

    G = nx.DiGraph()


    print("\nAdding occupation nodes...")
    for _, row in df_occupations.iterrows():
        G.add_node(
            row['id'],
            node_type='occupation',
            label=row['preferredLabel'],
            description=row['description'],
            isco_group=row.get('iscoGroup'),
            esco_code=row.get('code'),
            alt_labels=row.get('altLabels')
        )
    print(f"Added {df_occupations.shape[0]} occupation nodes.")


    print("\nAdding skill nodes...")
    for _, row in df_skills.iterrows():
        G.add_node(
            row['id'],
            node_type='skill',
            label=row['preferredLabel'],
            description=row['description'],
            skill_type_from_skills_file=row.get('skillType'), # Renaming to avoid clash with relations 'skillType'
            reuse_level=row.get('reuseLevel'),
            alt_labels=row.get('altLabels')
        )
    print(f"Added {df_skills.shape[0]} skill nodes.")

    #Add edges between occupations and skills using the relations file
    print("\nAdding occupation-skill relationships as edges...")
    edges_added = 0
    edges_skipped_missing_node = 0



    occupation_uri_col = 'occupationUri'
    skill_uri_col = 'skillUri'
    relation_type_col = 'relationType'
    skill_type_relation_col = 'skillType' # This is the skillType from the relations file

    for _, row in df_relations.iterrows():
        occupation_uri = row[occupation_uri_col]
        skill_uri = row[skill_uri_col]
        relation_type = row[relation_type_col]
        skill_type_from_relation = row[skill_type_relation_col]

        if G.has_node(occupation_uri) and G.has_node(skill_uri):
            G.add_edge(
                occupation_uri,
                skill_uri,
                relation_type=relation_type,
                skill_type_of_relation=skill_type_from_relation
            )
            edges_added += 1
        else:
            edges_skipped_missing_node +=1


    print(f"Added {edges_added} occupation-skill edges.")
    if edges_skipped_missing_node > 0:
        print(f"Skipped {edges_skipped_missing_node} edges due to missing occupation or skill nodes. This might indicate inconsistencies if the URI schemes don't perfectly match or if your occupation/skill files are subsets of a larger ESCO version used to create the relations file.")


    print("\n--- Knowledge Graph Summary ---")
    print(f"Total nodes: {G.number_of_nodes()}")
    print(f"Total edges: {G.number_of_edges()}")

    if edges_added > 0 and df_relations.shape[0] > 0 :
        # Attempt to show an example relationship
        # Find first valid relationship from the relations file that was successfully added
        for _, rel_row in df_relations.iterrows():
            occ_uri_example = rel_row[occupation_uri_col]
            skl_uri_example = rel_row[skill_uri_col]
            if G.has_edge(occ_uri_example, skl_uri_example):
                print(f"\nExample: Skills related to occupation '{G.nodes[occ_uri_example].get('label', occ_uri_example)}' (URI: {occ_uri_example}):")
                edge_data = G.get_edge_data(occ_uri_example, skl_uri_example)
                print(f"  - Skill: '{G.nodes[skl_uri_example].get('label', skl_uri_example)}' (URI: {skl_uri_example}), Relation Type: {edge_data.get('relation_type')}, Skill Type of Relation: {edge_data.get('skill_type_of_relation')}")
                break # Show only one example
    else:
        print("\nNo occupation-skill edges were added or no relations to sample from. Check your relations file and node identifiers if this is unexpected.")

    return G

# Build the graph
esco_kg = build_esco_knowledge_graph()

# If you want to save the graph (optional):
if esco_kg:
    try:
        nx.write_graphml(esco_kg, "esco_knowledge_graph.graphml")
        print("\nKnowledge graph saved to esco_knowledge_graph.graphml")
    except Exception as e:
        print(f"\nError saving graph to GraphML: {e}")

Successfully loaded all CSV files.

Adding occupation nodes...
Added 3039 occupation nodes.

Adding skill nodes...
Added 13939 skill nodes.

Adding occupation-skill relationships as edges...
Added 129004 occupation-skill edges.

--- Knowledge Graph Summary ---
Total nodes: 16978
Total edges: 128987

Example: Skills related to occupation 'directeur technique/directrice technique' (URI: http://data.europa.eu/esco/occupation/00030d09-2b3a-4efd-87cc-c4ea39d27c34):
  - Skill: 'techniques théâtrales' (URI: http://data.europa.eu/esco/skill/fed5b267-73fa-461d-9f69-827c78beb39d), Relation Type: essential, Skill Type of Relation: knowledge

Knowledge graph saved to esco_knowledge_graph.graphml


In [ ]:
# Exploring the graph


occupation_uri = "http://data.europa.eu/esco/occupation/00030d09-2b3a-4efd-87cc-c4ea39d27c34"
if esco_kg and occupation_uri in esco_kg:
    skills = [nbr for nbr in esco_kg.successors(occupation_uri) if esco_kg.nodes[nbr]['node_type'] == 'skill']
    print(f"Skills for occupation {occupation_uri}:")
    for skill in skills:
        print(f"- {esco_kg.nodes[skill]['label']}")
else:
    print("Occupation not found in the graph.")

Skills for occupation http://data.europa.eu/esco/occupation/00030d09-2b3a-4efd-87cc-c4ea39d27c34:
- techniques théâtrales
- organiser des répétitions
- rédiger une évaluation des risques sur la représentation d'une production artistique
- assurer la coordination avec les services créatifs
- s’adapter aux exigences créatives d’artistes
- négocier des questions de santé et de sécurité avec des tiers
- adapter le travail des concepteurs à la salle de spectacle
- promouvoir la santé et la sécurité
- coordonner les équipes techniques dans des productions artistiques
- rédiger des devis techniques


In [ ]:
[node_data.get('description') for node_id, node_data in esco_kg.nodes(data=True) if node_data.get('node_type') in ('occupation')][:5]

['Les directeurs techniques/directrices techniques donnent corps aux visions artistiques des créateurs dans le respect de contraintes techniques. Ils/elles coordonnent les opérations des différentes unités de production, telles que la scène, les costumes, le son, l’éclairage et le maquillage. Ils/elles adaptent le prototype et étudient la faisabilité, la mise en œuvre, l’exploitation et le suivi technique du projet artistique. Ils/elles sont également responsables du matériel scénique et de l’équipement technique.',
 'Les opérateurs de tréfilerie installent et font fonctionner des machines à tréfiler (appelées «bancs de tréfilage» ou «tréfileuses») pour les métaux ferreux et non ferreux destinés à la production de fils, de barres, de conduits, de profilés creux et de tuyaux rigides de forme spécifique, en réduisant la section transversale et en étirant la matière à travailler au moyen de diverses matrices à emboutir.',
 'Les contrôleurs qualité des instruments de précision veillent à c

In [ ]:
esco_skill_labels_to_uri = {
        str(data.get('label', '')).lower(): node_id
        for node_id, data in esco_kg.nodes(data=True)
        if data.get('node_type') == 'skill' and data.get('label')
    }
print(f"Created mapping for {len(esco_skill_labels_to_uri)} skill labels.")

esco_occupation_labels_to_uri = {
        str(data.get('label', '')).lower(): node_id
        for node_id, data in esco_kg.nodes(data=True)
        if data.get('node_type') == 'occupation' and data.get('label')
    }
print(f"Created mapping for {len(esco_occupation_labels_to_uri)} occupation labels.")


Created mapping for 13934 skill labels.
Created mapping for 3038 occupation labels.


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/new_everything_dataset.csv")

df.columns


/tmp/ipython-input-7-703060930.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/new_everything_dataset.csv")


Index(['Unnamed: 0.1', 'Unnamed: 0', 'Id_application', 'Date', 'Id_candidate',
       'Id_vacancy', 'Civilité', 'Ville', 'status', 'Motif de refus', 'Métier',
       'Tranche d'age', 'Type de contrat',
       'Prétentions salariales mensuelles fixe net', 'Permis de conduire',
       'Mobilité', 'Code postal', 'Region', 'name', 'fileName', 'nationality',
       'permis', 'address', 'experiences', 'text', 'gender', 'postcode',
       'jobTitle', 'vacancy_text', 'candidate_text', 'label'],
      dtype='object')

In [ ]:
df['label'].value_counts()

,count
label,
0.0,34720
1.0,417


In [ ]:
df['vacancy_text'][0]

'Titre: Chargé de relation Clientèle (H/F) - Marcq-en-Baroeul\nFonction: Commerce\nVille et Region: Marcq-en-Baroeul, Nord pas de Calais\nType de poste: CDI\nDescriptif de l\'offre: KILOUTOU a créé un nouveau service pour ses clients. Cette cellule a pour vocation de gérer tous les appels non décrochés en agences afin de garantir une réponse à nos clients professionnels ou particuliers.   Dans le cadre du développement de cette nouvelle cellule, nous recherchons des\xa0Chargés de relation clientèle (H/F).   Ton quotidien professionnel ?   Nous te proposons donc de rejoindre l\'équipe Kiloutou (et quelle équipe\xa0!) pour intégrer notre centre de relation client\xa0: un nouveau QG (que nous en sommes sûrs, tu ne voudras plus le quitter).   En véritable commercial sédentaire polyvalent, tu y exerces tout ton talent\xa0: tu réceptionnes les appels des clients (particuliers et professionnels), analyses leurs besoins, leur apportes la réponse ou la solution la plus adaptée et assures la lia

In [ ]:
df['candidate_text'][0]

"DELEPLANQUE Aurele COMPETENCES Capacité d’analyse, d’écoute, de synthèse, autonomie, gestion des priorités Goût du challenge Maitrise du pack office CONTACT 18 rue des ormeaux 59175 Vendeville 06 58 43 34 20 mailto: SITUATION Née le 03/11/1984 Permis B et véhicule POINTS FORTS Réactive Dynamique Rigoureuse Capacité d’adaptation Gestion du stress INTERETS Voyages Lecture Cuisine ASSISTANTE ADV PARCOURS PROFESSIONNEL SECURIRACKFRANCE–ASSISTANTEADMINISTRATIVEETCO MMERCIALE Depuis septembre 2020 Création de contrats, de devis Gestion administrative du personnel Accueil physique et téléphonique-courriers Préparation à la comptabilité Facturation et relance (clients-fournisseurs) Préparation de salons professionnels Organisation du planning des opérations Réservations des logements lors des chantiers Gestion du stock Gestion des expéditions Organisation et suivi des transports LOXAMA CCE SS-FRETIN–RESPONSABLEDELOCATION/BTOB 2018 – 2020 Contrats clients : offres de prix, édition des factures

In [ ]:
# How many candidate texts have "competences" in them?

df['candidate_text'].str.contains("competences", case=False).sum()

np.int64(9029)

In [ ]:
# Okay and wnat about the ones that dont have competences in them?

df['candidate_text'][~df['candidate_text'].fillna('').str.contains("competences", case=False)].head(10)[1]

"ALBICE Yohann 2 rue d'Alembert 93000 BOBIGNY 24 ans Permis B Tel : 06 46 80 71 14 mailto: Email : Expérience professionnelle 2022 Intérim 1 an, Setha – Ouvrier-Bobigny 2021 Intérim 4 mois Colbert location – Responsable de parc - Torcy 2020 Intérim 6 mois, Emulithe – Manœuvre - Bobigny 2020 CDI 1 mois, Lidl – Équipier polyvalent - Bondy 2019 Intérim 3 mois, Agent exploitation logistique Blanc-Mesnil 2019 CDI 1 mois, Tang-Frères – Équipier polyvalent Bussy-Saint-Georges 2018 CDD 6 mois, Kiloutou – SA Aide Préparateur-Technique Gennevilliers 2017 Stage ATECH ELEC Systems Neuilly Sur Seine 2016 Stage Campanille – Maintenance Électrique Roissy 2015 Stage Assurance Maladie Seine Saint Denis 2014 Stage Campanille– Maintenance Electrique Roissy Formation Terminale Bac Pro Électrotechnique Énergie Équipement Communication LP Paul le Rolland – DRANCY 6h / semaine : Mise en service 6h / semaine : Programmation 2 nde Bac Pro Électrotechnique Énergie Équipement Communicant LP Paul le Rolland – DRA

In [ ]:
# Extracting "esco titles" of the candidate from the "experience column"

import ast

def extract_non_blank_esco_titles_ast(experiences_str):
    """
    Extracts non-blank 'escoTitle' values from a string representation of a list of dictionaries
    using ast.literal_eval to handle potentially non-strict JSON.

    Args:
        experiences_str: A string representing a list of dictionaries, where each
                         dictionary contains job experience information, including
                         'escoTitle'.

    Returns:
        A list of non-blank 'escoTitle' strings found in the input.
    """
    try:
        experiences_list = ast.literal_eval(experiences_str)
        esco_titles = [exp['escoTitle'] for exp in experiences_list if isinstance(exp, dict) and exp.get('escoTitle')]
        return esco_titles
    except (SyntaxError, ValueError):
        print(f"Error decoding string with ast.literal_eval: {experiences_str}")
        return []

# Apply the corrected function to the 'experiences' column
df['esco_titles_candidates'] = df['experiences'].apply(extract_non_blank_esco_titles_ast)

In [ ]:
df['esco_titles_candidates'].head()

,esco_titles_candidates
0,[responsable des transports et des infrastruct...
1,"[ouvrier brasseur-malteur, exploitante de parc..."
2,[]
3,"[mécanicien diéséliste, technicien de maintena..."
4,[jardinière paysagiste]


In [ ]:
df['esco_titles_candidates'].isnull().sum()

np.int64(0)

In [ ]:
for element in df['esco_titles_candidates']:
    print(type(element))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

In [ ]:
# converting the list of esco titles to a string representation
df['esco_titles_candidates'] = df['esco_titles_candidates'].apply(lambda x: str(x) if isinstance(x, list) else '')

# Delete the rows with empty esco_titles_candidate
df = df[~(df['esco_titles_candidates'].isnull() | (df['esco_titles_candidates'] == '[]'))]



In [ ]:
df['esco_titles_candidates'].head()

,esco_titles_candidates
0,['responsable des transports et des infrastruc...
1,"['ouvrier brasseur-malteur', 'exploitante de p..."
3,"['mécanicien diéséliste', 'technicien de maint..."
4,['jardinière paysagiste']
5,"['technicien de maintenance aéroportuaire', 'd..."


In [ ]:
df['esco_titles_candidates'] = df['esco_titles_candidates'].str.replace('[', '')
df['esco_titles_candidates'] = df['esco_titles_candidates'].str.replace(']', '')
df['esco_titles_candidates'] = df['esco_titles_candidates'].str.replace("'", '')

In [ ]:
df['esco_titles_candidates'].head()

,esco_titles_candidates
0,responsable des transports et des infrastructures
1,"ouvrier brasseur-malteur, exploitante de parc ..."
3,"mécanicien diéséliste, technicien de maintenan..."
4,jardinière paysagiste
5,"technicien de maintenance aéroportuaire, dessi..."


In [ ]:
def get_skills_for_occupation_title(occupation_title, graph, occupation_labels_to_uri):
    """
    Get skills for a single occupation title using both preferred and alternative labels
    """
    # Clean and lowercase the occupation title
    occupation_title = str(occupation_title).lower().strip()

    # Try to find URI using the label mapping
    occupation_uri = occupation_labels_to_uri.get(occupation_title)

    # If not found, try to find using alternative labels
    if not occupation_uri:
        for node_id, data in graph.nodes(data=True):
            if data.get('node_type') == 'occupation':
                alt_labels = str(data.get('alt_labels', '')).lower()
                if occupation_title in alt_labels:
                    occupation_uri = node_id
                    break

    if occupation_uri and occupation_uri in graph:
        # Get skills for this occupation
        skills = []
        for skill_uri in graph.successors(occupation_uri):
            if graph.nodes[skill_uri]['node_type'] == 'skill':
                skill_data = graph.nodes[skill_uri]
                skills.append({
                    'skill_label': skill_data['label'],
                    'skill_type': graph.get_edge_data(occupation_uri, skill_uri).get('skill_type_of_relation'),
                    'relation_type': graph.get_edge_data(occupation_uri, skill_uri).get('relation_type')
                })
        return skills
    return []

def map_candidate_occupations_to_skills(df, graph, occupation_labels_to_uri):
    """
    Map all occupations in esco_title_candidates to their corresponding skills
    """
    # Create a new column for skills
    df['candidate_occupation_skills'] = df['esco_titles_candidates'].apply(
        lambda x: [] if pd.isna(x) else [
            skill
            for occupation in str(x).split(', ')
            for skill in get_skills_for_occupation_title(occupation, graph, occupation_labels_to_uri)
        ]
    )
    return df

# Apply the mapping
df = map_candidate_occupations_to_skills(df, esco_kg, esco_occupation_labels_to_uri)

# Show an example of the results
print("\nExample of mapped skills for first row:")
if not df['candidate_occupation_skills'].empty:
    print(f"Occupation titles: {df['esco_titles_candidates'].iloc[0]}")
    print("\nCorresponding skills:")
    for skill in df['candidate_occupation_skills'].iloc[0]:
        print(f"- {skill['skill_label']} ({skill['skill_type']}, {skill['relation_type']})")


Example of mapped skills for first row:
Occupation titles: responsable des transports et des infrastructures

Corresponding skills:
- politique environnementale (knowledge, essential)
- droit de l’urbanisme (knowledge, essential)
- législation environnementale (knowledge, essential)
- itinéraires géographiques (knowledge, essential)
- statistiques (knowledge, essential)
- urbanisme (knowledge, essential)
- logiciel de système d'analyse statistique (knowledge, essential)
- ingénierie de la circulation (knowledge, essential)
- droit public (knowledge, essential)
- définir des modèles statistiques (skill/competence, essential)
- réaliser des études sur les transports urbains (skill/competence, essential)
- interpréter des éléments de culture visuelle (skill/competence, essential)
- appliquer des techniques d’analyse statistique (skill/competence, essential)
- promouvoir l’utilisation de transports durables (skill/competence, essential)
- analyser les coûts de transport (skill/competence,

In [ ]:
df['candidate_occupation_skills'][0]

[{'skill_label': 'politique environnementale',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'droit de l’urbanisme',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'législation environnementale',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'itinéraires géographiques',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'statistiques',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'urbanisme',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': "logiciel de système d'analyse statistique",
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'ingénierie de la circulation',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'droit public',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'définir des modèles statistiques',

In [ ]:
df['candidate_occupation_skills'].isnull().sum()

np.int64(0)

In [ ]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Id_application', 'Date', 'Id_candidate',
       'Id_vacancy', 'Civilité', 'Ville', 'status', 'Motif de refus', 'Métier',
       'Tranche d'age', 'Type de contrat',
       'Prétentions salariales mensuelles fixe net', 'Permis de conduire',
       'Mobilité', 'Code postal', 'Region', 'name', 'fileName', 'nationality',
       'permis', 'address', 'experiences', 'text', 'gender', 'postcode',
       'jobTitle', 'vacancy_text', 'candidate_text', 'label',
       'esco_titles_candidates', 'candidate_occupation_skills'],
      dtype='object')

In [ ]:
df_vac = pd.read_csv('vacancies.csv', sep=";")




In [ ]:
df_vac.shape

(13204, 16)

In [ ]:
df_vac.columns

Index(['Id', 'Titre', 'Région', 'Département', 'Ville', 'Fonction',
       'Date d'activation', 'Type de poste', 'Type de formation',
       'Salaire min (euro brut annuel)', 'Salaire max (euro brut annuel)',
       'Profil attendu', 'Expérience', 'Descriptif de l'offre', 'Actif',
       'Archivé'],
      dtype='object')

In [ ]:
# Merge df and df_vac on the vacancy ID
df = pd.merge(df, df_vac[['Id', 'Titre']], left_on='Id_vacancy', right_on='Id', how='left')

# Rename the 'Titre' column from df_vac to 'vacancy_title' and drop the redundant 'Id' column
df = df.rename(columns={'Titre': 'vacancy_title'}).drop('Id', axis=1)

print("Successfully added 'vacancy_title' column.")

Successfully added 'vacancy_title' column.


In [ ]:
df['vacancy_title']

,vacancy_title
0,Chargé de relation Clientèle (H/F) - Marcq-en-...
1,Mécanicien spécialisé (H/F) - Gennevilliers
2,Mécanicien Spécialisé (H/F) - Le Havre
3,Conducteur VL (H/F) - Lyon Sud
4,Mécanicien Spécialisé (H/F) - Saint Maximin
...,...
50388,Mécanicien (H/F) - Laval
50389,Conducteur PL (H/F) - Pérols
50390,Chef de projet recrutement (H/F)-Paris
50391,Chargé de Clientèle (H/F) - Vandoeuvre-lès-Nancy


In [ ]:
df['vacancy_title'].isnull().sum()

np.int64(0)

In [ ]:
# remove the location name from the vacancy title

df['vacancy_title'] = df['vacancy_title'].str.split('-').str[0]

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Tuple
import torch
from tqdm import tqdm

def prepare_occupation_data_optimized(graph):
    """
    Prepare occupation data from the knowledge graph for matching
    Returns both the occupation metadata and their combined texts
    """
    occupations = []
    occupation_texts = []

    for node_id, data in graph.nodes(data=True):
        if data.get('node_type') == 'occupation':
            label = data.get('label', '').lower()
            #description = data.get('description', '').lower()
            #alt_labels = str(data.get('alt_labels', '')).lower()

            text_for_matching = f"{label}" # f"{label} {alt_labels} {description}"

            occupations.append({
                'uri': node_id,
                'label': label
            })
            occupation_texts.append(text_for_matching)

    return occupations, occupation_texts

def find_matching_occupations_batch(vacancy_texts: List[str],
                                  occupation_embeddings: torch.Tensor,
                                  occupations: List[dict],
                                  model,
                                  batch_size: int = 32,
                                  top_k: int = 5) -> List[List[Tuple[str, float]]]:
    """
    Find matching occupations for multiple vacancy texts in batches using proper cosine similarity
    """
    all_matches = []

    # Normalize occupation embeddings once (for cosine similarity)
    occupation_embeddings_normalized = torch.nn.functional.normalize(occupation_embeddings, p=2, dim=1)

    # Process vacancies in batches
    for i in range(0, len(vacancy_texts), batch_size):
        batch = vacancy_texts[i:i + batch_size]

        # Encode batch of vacancy texts and normalize
        with torch.no_grad():
            vacancy_embeddings = model.encode(batch, convert_to_tensor=True)
            # Normalize vacancy embeddings (crucial for cosine similarity)
            vacancy_embeddings_normalized = torch.nn.functional.normalize(vacancy_embeddings, p=2, dim=1)

        # Calculate cosine similarity
        # cos_sim = a·b / (||a|| ||b||)
        # Since vectors are normalized, dot product gives us cosine similarity directly
        similarities = torch.mm(vacancy_embeddings_normalized, occupation_embeddings_normalized.t())

        # Get top-k matches for each vacancy in the batch
        batch_matches = []
        for sim_scores in similarities:
            top_indices = torch.topk(sim_scores, k=top_k).indices.cpu().numpy()
            matches = [(occupations[idx]['label'], sim_scores[idx].item())
                      for idx in top_indices]
            batch_matches.append(matches)

        all_matches.extend(batch_matches)

    return all_matches

# Set up the model and device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
model.to(device)
print("Device: ", device)

# Prepare and pre-compute occupation embeddings
print("Preparing occupation data...")
occupations, occupation_texts = prepare_occupation_data_optimized(esco_kg)

print("Pre-computing occupation embeddings...")
with torch.no_grad():
    occupation_embeddings = model.encode(occupation_texts,
                                       convert_to_tensor=True,
                                       batch_size=32,
                                       show_progress_bar=True)

# Process all vacancies in batches
print("Processing vacancies...")
vacancy_texts = df['vacancy_title'].fillna('').tolist()
matched_occupations = find_matching_occupations_batch(
    vacancy_texts,
    occupation_embeddings,
    occupations,
    model,
    batch_size=32
)

# Add results to dataframe
df['matched_occupations'] = matched_occupations
df['esco_title_vacancy'] = df['matched_occupations'].apply(lambda x: x[0][0] if x else None)
df['match_score'] = df['matched_occupations'].apply(lambda x: x[0][1] if x else None)

# Map to skills (this part remains the same)
df['vacancy_occupation_skills'] = df['esco_title_vacancy'].apply(
    lambda x: get_skills_for_occupation_title(x, esco_kg, esco_occupation_labels_to_uri)
    if not pd.isna(x) else []
)

Device:  cuda
Preparing occupation data...
Pre-computing occupation embeddings...


Batches:   0%|          | 0/95 [00:00<?, ?it/s]

Processing vacancies...


In [ ]:
df[['matched_occupations', 'esco_title_vacancy', 'match_score', 'vacancy_occupation_skills']].head(10)

,matched_occupations,esco_title_vacancy,match_score,vacancy_occupation_skills
0,"[(analyste coûts, 0.6445102095603943), (respon...",analyste coûts,0.644510,[{'skill_label': 'procédures des services fina...
1,[(ingénieur mécanicien/ingénieure mécanicienne...,ingénieur mécanicien/ingénieure mécanicienne,0.762565,"[{'skill_label': 'simulation sur ordinateur', ..."
2,[(mécanicien de précision/mécanicienne de préc...,mécanicien de précision/mécanicienne de précision,0.697825,"[{'skill_label': 'mécanique de précision', 'sk..."
3,[(conducteur de niveleuse/conductrice de nivel...,conducteur de niveleuse/conductrice de niveleuse,0.699769,"[{'skill_label': 'outils mécaniques', 'skill_t..."
4,[(mécanicien de précision/mécanicienne de préc...,mécanicien de précision/mécanicienne de précision,0.697825,"[{'skill_label': 'mécanique de précision', 'sk..."
5,"[(chef cuisinier/cheffe cuisinière, 0.82330083...",chef cuisinier/cheffe cuisinière,0.823301,[{'skill_label': 'systèmes de suivi du gaspill...
6,[(directeur des licences/directrices des licen...,directeur des licences/directrices des licences,0.655164,[{'skill_label': 'droit de la propriété intell...
7,"[(spécialiste des technologies d’assistance, 0...",spécialiste des technologies d’assistance,0.729259,[{'skill_label': 'technologies d’aide à l’ense...
8,[(directeur des licences/directrices des licen...,directeur des licences/directrices des licences,0.655164,[{'skill_label': 'droit de la propriété intell...
9,[(mécanicien de précision/mécanicienne de préc...,mécanicien de précision/mécanicienne de précision,0.852970,"[{'skill_label': 'mécanique de précision', 'sk..."


In [ ]:
df[['vacancy_title', 'esco_title_vacancy', 'vacancy_occupation_skills']]

,vacancy_title,esco_title_vacancy,vacancy_occupation_skills
0,Chargé de relation Clientèle (H/F),analyste coûts,[{'skill_label': 'procédures des services fina...
1,Mécanicien spécialisé (H/F),ingénieur mécanicien/ingénieure mécanicienne,"[{'skill_label': 'simulation sur ordinateur', ..."
2,Mécanicien Spécialisé (H/F),mécanicien de précision/mécanicienne de précision,"[{'skill_label': 'mécanique de précision', 'sk..."
3,Conducteur VL (H/F),conducteur de niveleuse/conductrice de niveleuse,"[{'skill_label': 'outils mécaniques', 'skill_t..."
4,Mécanicien Spécialisé (H/F),mécanicien de précision/mécanicienne de précision,"[{'skill_label': 'mécanique de précision', 'sk..."
...,...,...,...
50388,Mécanicien (H/F),ingénieur mécatronicien/ingénieure mécatronici...,"[{'skill_label': 'principes d’ingénierie', 'sk..."
50389,Conducteur PL (H/F),conducteur de niveleuse/conductrice de niveleuse,"[{'skill_label': 'outils mécaniques', 'skill_t..."
50390,Chef de projet recrutement (H/F),directeur des ressources humaines/directrice d...,"[{'skill_label': 'législation sociale', 'skill..."
50391,Chargé de Clientèle (H/F),analyste coûts,[{'skill_label': 'procédures des services fina...


In [ ]:
# Save the DataFrame
df.to_csv('/content/drive/MyDrive/new_everything_esco_better.csv', index=False)


In [ ]:
df.columns

Index(['Unnamed: 0', 'Id_application', 'Date', 'Id_candidate', 'Id_vacancy',
       'Civilité', 'Ville', 'status', 'Motif de refus', 'Métier',
       'Tranche d'age', 'Type de contrat',
       'Prétentions salariales mensuelles fixe net', 'Permis de conduire',
       'Mobilité', 'Code postal', 'Region', 'name', 'fileName', 'nationality',
       'permis', 'address', 'experiences', 'text', 'gender', 'postcode',
       'jobTitle', 'vacancy_text', 'candidate_text', 'label', 'gender_number',
       'esco_titles_candidates', 'candidate_occupation_skills',
       'matched_occupations', 'esco_title_vacancy', 'match_score',
       'vacancy_occupation_skills'],
      dtype='object')

In [ ]:
# Let's see what we have for candidates

df[['candidate_text', 'esco_titles_candidates', 'candidate_occupation_skills']].head(3)

,candidate_text,esco_titles_candidates,candidate_occupation_skills
0,DELEPLANQUE Aurele COMPETENCES Capacité d’anal...,responsable des transports et des infrastructures,"[{'skill_label': 'politique environnementale',..."
1,ALBICE Yohann 2 rue d'Alembert 93000 BOBIGNY 2...,"ouvrier brasseur-malteur, exploitante de parc ...","[{'skill_label': 'échelles de température', 's..."
3,CONDUCTEUR DÉMONSTRATEUR PL CONTACT 553 CHEMIN...,jardinière paysagiste,[{'skill_label': 'législation environnementale...


In [ ]:
df[['candidate_text', 'esco_titles_candidates', 'candidate_occupation_skills']].isnull().sum()

candidate_text                 0
esco_titles_candidates         0
candidate_occupation_skills    0
dtype: int64

In [ ]:
# What do we have for vacancies?

df[['vacancy_text', 'matched_occupations', 'esco_title_vacancy', 'vacancy_occupation_skills']].head(3)

,vacancy_text,matched_occupations,esco_title_vacancy,vacancy_occupation_skills
0,Titre: Chargé de relation Clientèle (H/F) - Ma...,[(agent d’information de centre de contact cli...,agent d’information de centre de contact clien...,"[{'skill_label': 'connaissance du client', 'sk..."
1,Titre: Mécanicien spécialisé (H/F) - Gennevill...,[(technicien en filature/technicienne en filat...,technicien en filature/technicienne en filature,"[{'skill_label': 'techniques de filature', 'sk..."
3,Titre: Conducteur VL (H/F) - Lyon Sud\nFonctio...,"[(conseiller viticole/conseillère viticole, 0....",conseiller viticole/conseillère viticole,"[{'skill_label': 'types de vin', 'skill_type':..."


In [ ]:
df['matched_occupations'][0]

[('agent d’information de centre de contact clients/agente d’information de centre de contact clients',
  0.5015677213668823),
 ('responsable de l’approvisionnement en technologies de l’information et de la communication',
  0.4922410845756531),
 ('agent de centre d’appels/agente de centre d’appels', 0.4859301447868347),
 ('agent de réservation/agente de réservation', 0.46335867047309875),
 ('responsable de la relation clientèle', 0.4630245268344879)]

In [ ]:
df['vacancy_occupation_skills'].head()[0]

[{'skill_label': 'connaissance du client',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'caractéristiques des produits',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'service clients',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'caractéristiques des services',
  'skill_type': 'knowledge',
  'relation_type': 'essential'},
 {'skill_label': 'garantir la satisfaction des clients',
  'skill_type': 'skill/competence',
  'relation_type': 'essential'},
 {'skill_label': 'communiquer avec des clients',
  'skill_type': 'skill/competence',
  'relation_type': 'essential'},
 {'skill_label': 'utiliser des bases de données',
  'skill_type': 'skill/competence',
  'relation_type': 'essential'},
 {'skill_label': 'répondre à des appels entrants',
  'skill_type': 'skill/competence',
  'relation_type': 'essential'},
 {'skill_label': 'tenir des registres des interactions avec des clients',
  'skill_type': 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/Inasoft/2025_intern_work/new_everything_esco_better.csv')


/tmp/ipython-input-20-535010149.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Data/Inasoft/2025_intern_work/new_everything_esco_better.csv')


In [ ]:
df.drop(['candidate_text'], axis=1, inplace=True)

In [ ]:
df.drop(['vacancy_text'], axis=1, inplace=True)

In [ ]:
# Split the string of candidate ESCO titles and take the first one as the latest
df['esco_title_candidates_latest'] = df['esco_titles_candidates'].str.split(',').str[0]

In [ ]:
def create_candidate_text(esco_titles, skills_str):
    """Create French text for candidate experience and skills"""
    try:
        # Parse skills from string
        skills_list = eval(skills_str) if pd.notna(skills_str) else []
        skills_text = ", ".join([skill['skill_label'] for skill in skills_list])

        # Create French text
        text = f"J'ai travaillé dans les domaines suivants : {esco_titles}. "
        if skills_text:
            text += f"Je possède les compétences suivantes : {skills_text}."
        return text
    except:
        return esco_titles

def create_vacancy_text(esco_title, skills_str):
    """Create French text for vacancy requirements"""
    try:
        # Parse skills from string
        skills_list = eval(skills_str) if pd.notna(skills_str) else []
        skills_text = ", ".join([skill['skill_label'] for skill in skills_list])

        # Create French text
        text = f"Le poste recherché est : {esco_title}. "
        if skills_text:
            text += f"Les compétences requises sont : {skills_text}."
        return text
    except:
        return esco_title

# Create the sentence pairs
df['candidate_text'] = df.apply(
    lambda row: create_candidate_text(
        row['esco_title_candidates_latest'],
        row['candidate_occupation_skills']
    ),
    axis=1
)

df['vacancy_text'] = df.apply(
    lambda row: create_vacancy_text(
        row['esco_title_vacancy'],
        row['vacancy_occupation_skills']
    ),
    axis=1
)

# Create final dataset for training
sentence_pairs = df[['candidate_text', 'vacancy_text']].copy()

# Print example to verify format
print("Example Pair:")
print("\nCandidate (sentence1):")
print(sentence_pairs['candidate_text'].iloc[0])
print("\nVacancy (sentence2):")
print(sentence_pairs['vacancy_text'].iloc[0])

Example Pair:

Candidate (sentence1):
J'ai travaillé dans les domaines suivants : responsable des transports et des infrastructures. Je possède les compétences suivantes : politique environnementale, droit de l’urbanisme, législation environnementale, itinéraires géographiques, statistiques, urbanisme, logiciel de système d'analyse statistique, ingénierie de la circulation, droit public, définir des modèles statistiques, réaliser des études sur les transports urbains, interpréter des éléments de culture visuelle, appliquer des techniques d’analyse statistique, promouvoir l’utilisation de transports durables, analyser les coûts de transport, analyser des données sur l’environnement, analyser des données de tests, réglementer la circulation, analyser la configuration du trafic routier, mener des études environnementales, étudier les flux de circulation, analyser des études sur les transports, analyser des réseaux d’entreprises de transport, préparer des graphiques de données, communiquer

In [ ]:
df.to_csv('/content/drive/MyDrive/new_everything_esco_better.csv', index=False)